In [3]:
import numpy as np

In [4]:
def hex2bin(c):
    return np.binary_repr(int(c, base=16), width=4)

def decode2bin(s):
    num = ""
    for c in s:
        num += hex2bin(c)
    return num

def pack(arr, wordsize=8):
    npack = wordsize - len(arr)
    return np.hstack([np.zeros(npack, dtype=int), arr])

def pack8(arr):
    return np.packbits(pack(arr, wordsize=8))[0]

def pack16(arr):
    bytearr = np.packbits(pack(arr, wordsize=16))
    return 256 * bytearr[0] + bytearr[1]

def arr2str(arr):
    return ''.join([f"{i}" for i in arr])

In [6]:
test_str = "D2FE28"
decode2bin(test_str)

'110100101111111000101000'

In [7]:
decode2bin("38006F45291200")

'00111000000000000110111101000101001010010001001000000000'

In [124]:
def mainparser(binstr):
    versum = 0
    if len(binstr) > 6:
        ver = int(binstr[:3], base=2)
        typ = int(binstr[3:6], base=2)
        print(f"ver = {ver}, typ = {typ}")
        versum += ver
        binstr = binstr[6:]
        if typ == 4:
            binstr, val = literal(binstr)
            print(f"literal returned {val}")
        else:
            if binstr[0] == "0":
                subpacketlen = int(binstr[1:16], 2)
                binstr = binstr[16:]
                subpackets = binstr[:subpacketlen]
                binstr = binstr[subpacketlen:]
                subversum = 0
                while True:
                    v, subpackets = mainparser(subpackets)
                    subversum += v
                    if len(subpackets) < 6:
                        break
                versum += subversum
            else:
                subcount = int(binstr[1:12], base=2)
                binstr = binstr[12:]
                subversum = 0
                for _ in range(subcount):
                    v, binstr = mainparser(binstr)
                    subversum += v
                versum += subversum
    return versum, binstr

def literal(binstr):
    chunks = []
    chunking = True
    while chunking:
        chunks += [binstr[:5]]
        binstr = binstr[5:]
        if chunks[-1][0] == '0':
            chunking = False
    l = int(''.join([x[1:] for x in chunks]), 2)
    return binstr, l

In [125]:
mainparser(decode2bin(test_str))

ver = 6, typ = 4
literal returned 2021


(6, '000')

In [126]:
mainparser(decode2bin("38006F45291200"))

ver = 1, typ = 6
ver = 6, typ = 4
literal returned 10
ver = 2, typ = 4
literal returned 20


(9, '0000000')

In [127]:
mainparser(decode2bin("EE00D40C823060"))

ver = 7, typ = 3
ver = 2, typ = 4
literal returned 1
ver = 4, typ = 4
literal returned 2
ver = 1, typ = 4
literal returned 3


(14, '00000')

In [128]:
mainparser(decode2bin("8A004A801A8002F478"))

ver = 4, typ = 2
ver = 1, typ = 2
ver = 5, typ = 2
ver = 6, typ = 4
literal returned 15


(16, '000')

In [129]:
mainparser(decode2bin("620080001611562C8802118E34"))

ver = 3, typ = 0
ver = 0, typ = 0
ver = 0, typ = 4
literal returned 10
ver = 5, typ = 4
literal returned 11
ver = 1, typ = 0
ver = 0, typ = 4
literal returned 12
ver = 3, typ = 4
literal returned 13


(12, '00')

In [130]:
mainparser(decode2bin("C0015000016115A2E0802F182340"))

ver = 6, typ = 0
ver = 0, typ = 0
ver = 0, typ = 4
literal returned 10
ver = 6, typ = 4
literal returned 11
ver = 4, typ = 0
ver = 7, typ = 4
literal returned 12
ver = 0, typ = 4
literal returned 13


(23, '000000')

In [131]:
mainparser(decode2bin("A0016C880162017C3686B18A3D4780"))

ver = 5, typ = 0
ver = 1, typ = 0
ver = 3, typ = 0
ver = 7, typ = 4
literal returned 6
ver = 6, typ = 4
literal returned 6
ver = 5, typ = 4
literal returned 12
ver = 2, typ = 4
literal returned 15
ver = 2, typ = 4
literal returned 15


(31, '0000000')

In [132]:
with open("input.txt", 'r') as fp:
    data = fp.readline().strip()
mainparser(decode2bin(data))

ver = 1, typ = 0
ver = 1, typ = 0
ver = 2, typ = 4
literal returned 206
ver = 0, typ = 4
literal returned 176
ver = 4, typ = 4
literal returned 14
ver = 0, typ = 3
ver = 0, typ = 4
literal returned 12148392
ver = 5, typ = 4
literal returned 68454822
ver = 6, typ = 1
ver = 3, typ = 6
ver = 3, typ = 4
literal returned 2967250
ver = 6, typ = 4
literal returned 95
ver = 7, typ = 4
literal returned 1247339414
ver = 6, typ = 1
ver = 4, typ = 4
literal returned 4731280
ver = 1, typ = 7
ver = 6, typ = 4
literal returned 49792
ver = 3, typ = 4
literal returned 3454
ver = 3, typ = 4
literal returned 732272
ver = 5, typ = 1
ver = 7, typ = 5
ver = 4, typ = 0
ver = 5, typ = 4
literal returned 10
ver = 3, typ = 4
literal returned 13
ver = 1, typ = 4
literal returned 6
ver = 6, typ = 0
ver = 3, typ = 4
literal returned 2
ver = 5, typ = 4
literal returned 8
ver = 1, typ = 4
literal returned 11
ver = 5, typ = 4
literal returned 106
ver = 3, typ = 2
ver = 4, typ = 4
literal returned 11
ver = 1, typ = 4


(904, '0000000')